# Aggregate, Annotate, Normalize, and Feature Select

This notebook will run all these above operations on a single-cell file or well-aggregated file.

We will have the option to run or not 

In [1]:
mport pandas as pd
import numpy as np
import pycytominer
import easygui as eg
import os
import sys

path_scripts = r"C:\Users\Fer\Documents\GitHub"
sys.path.append(path_scripts)

from scripts_notebooks_fossa.profiles import generate_profiles

%load_ext autoreload
%autoreload 2

# 0) Inputs

In [30]:
profile = eg.fileopenbox(msg="Choose a file with samples and their features", default=r"D:")
print('Filename', profile)

project_name = input('Provide the name of this project:')
print('Project name:', project_name)

metadata_question = input(r"If you need to annotate your dataset with an external file, write yes and press enter. If already annotated, answer no and press enter.")
metadata_answer = generate_profiles.stringToBool(metadata_question)

if metadata_answer:
  platemap = eg.fileopenbox(msg="Choose a map (csv file) with plates names and metadata filenames", default=r"G:")
  platemap_path = os.path.split(platemap)[0]
  print('Platemap file selected', platemap)
  barcode_df = pd.read_csv(platemap)


Filename D:\2022_09_09_LiveCellPainting_fossa_Cimini\workspace\profiles\2022_03_24_Acidification_LT\2022_03_24_Acidification_LTsingle_cells.csv
Project name: 2022_03_24_Acidification_LT
Platemap file selected D:\2022_09_09_LiveCellPainting_fossa_Cimini\workspace\metadata\platemaps\2022_03_24_Acidification_LT\barcode_platemap.csv


## 1) Import extracted features file (single cell or well-aggregated)

In [56]:
df = pd.read_csv(profile)
df.head()

cells_that_run = []
cells_that_run.append(project_name)

,Unnamed: 0,Metadata_ImageNumber,Metadata_Plate,Metadata_Site,Metadata_Well,Metadata_TableNumber,Metadata_ObjectNumber_cytoplasm,Cytoplasm_AreaShape_Area,Cytoplasm_AreaShape_BoundingBoxArea,Cytoplasm_AreaShape_BoundingBoxMaximum_X,...,Nuclei_Texture_Variance_OrigLT_10_02_256,Nuclei_Texture_Variance_OrigLT_10_03_256,Nuclei_Texture_Variance_OrigLT_20_00_256,Nuclei_Texture_Variance_OrigLT_20_01_256,Nuclei_Texture_Variance_OrigLT_20_02_256,Nuclei_Texture_Variance_OrigLT_20_03_256,Nuclei_Texture_Variance_OrigLT_5_00_256,Nuclei_Texture_Variance_OrigLT_5_01_256,Nuclei_Texture_Variance_OrigLT_5_02_256,Nuclei_Texture_Variance_OrigLT_5_03_256
0,0,1,220324_134819_Plate_1,1,C10,910470619,1,12277,24750,304,...,172.188039,191.776783,206.014046,180.528021,192.152952,159.039568,206.944097,201.633954,198.510129,204.705950
1,1,1,220324_134819_Plate_1,1,C10,910470619,2,14291,29808,455,...,48.931053,54.246853,24.282060,28.964805,59.981566,77.895028,70.749460,58.278640,50.693477,47.509867
2,2,1,220324_134819_Plate_1,1,C10,910470619,3,7964,31980,195,...,102.704277,90.249529,95.657071,63.552313,104.850935,111.064484,117.455004,121.552540,120.074147,105.702546
3,3,1,220324_134819_Plate_1,1,C10,910470619,4,20542,44590,290,...,13.866668,15.379843,15.547724,19.049825,11.638127,15.130115,14.145880,14.388920,16.262809,14.580236
4,4,1,220324_134819_Plate_1,1,C10,910470619,5,26929,52664,724,...,23.833673,25.477788,25.495137,28.227327,27.681037,34.870136,26.611093,27.155324,25.444177,24.750500


### 1.a) Count number of cells in each well 

In [32]:
object_count = (
                df.groupby(["Metadata_Plate", "Metadata_Well"])["Nuclei_Number_Object_Number"]
                .count()
                .reset_index()
                .rename({"Nuclei_Number_Object_Number": "Metadata_cell_count"}, axis="columns")
            )

## 2) Generate profile

### 2A) Aggregate

- Run the next cell to list the metadata columns, and copy them to choose from which metadata information you'd like to join the rows on (strata)

- If using **already aggregated data by plates and wells**, skip to 2B.

In [4]:
pycytominer.cyto_utils.infer_cp_features(df, metadata=True)

['Metadata_ImageNumber',
 'Metadata_Plate',
 'Metadata_Site',
 'Metadata_Well',
 'Metadata_TableNumber',
 'Metadata_ObjectNumber_cytoplasm',
 'Metadata_Cytoplasm_Parent_Cells',
 'Metadata_Cytoplasm_Parent_Nuclei',
 'Metadata_ObjectNumber_cells',
 'Metadata_ObjectNumber']

- As a default we are using **Metadata_Plate and Metadata_Well** to join the rows, using the **'median'** operation. 

In [57]:
df = pycytominer.aggregate(df, 
                              strata=['Metadata_Plate', 'Metadata_Well'], 
                              operation='median')
df.head()

,Metadata_Plate,Metadata_Well,Cytoplasm_AreaShape_Area,Cytoplasm_AreaShape_BoundingBoxArea,Cytoplasm_AreaShape_BoundingBoxMaximum_X,Cytoplasm_AreaShape_BoundingBoxMaximum_Y,Cytoplasm_AreaShape_BoundingBoxMinimum_X,Cytoplasm_AreaShape_BoundingBoxMinimum_Y,Cytoplasm_AreaShape_Center_X,Cytoplasm_AreaShape_Center_Y,...,Nuclei_Texture_Variance_OrigLT_10_02_256,Nuclei_Texture_Variance_OrigLT_10_03_256,Nuclei_Texture_Variance_OrigLT_20_00_256,Nuclei_Texture_Variance_OrigLT_20_01_256,Nuclei_Texture_Variance_OrigLT_20_02_256,Nuclei_Texture_Variance_OrigLT_20_03_256,Nuclei_Texture_Variance_OrigLT_5_00_256,Nuclei_Texture_Variance_OrigLT_5_01_256,Nuclei_Texture_Variance_OrigLT_5_02_256,Nuclei_Texture_Variance_OrigLT_5_03_256
0,220324_134819_Plate_1,C10,13267.0,26796.0,696.0,578.0,529.0,402.0,618.768747,490.928433,...,52.592866,55.072915,55.256215,58.614150,58.262758,61.851628,56.082364,53.174189,53.351096,50.939043
1,220324_134819_Plate_1,C8,18487.0,36134.0,724.5,571.0,531.5,385.5,624.512243,475.329915,...,101.579659,104.833366,102.590733,110.913864,102.041391,113.269072,103.157839,102.426720,106.551106,105.263549
2,220324_134819_Plate_1,C9,13181.0,25584.0,678.0,529.0,488.0,352.0,592.016281,425.931690,...,18.157969,18.174451,19.227563,19.779993,18.090324,21.496804,18.622563,18.108056,19.082278,18.135697
3,220324_134819_Plate_1,D10,12475.0,23460.0,688.0,562.0,535.0,413.0,610.432063,486.678547,...,106.814632,102.613806,108.564186,109.454023,110.896811,113.285510,105.929430,103.481967,109.908506,104.247975
4,220324_134819_Plate_1,D8,17496.0,32831.0,674.0,546.0,471.5,381.5,566.840822,460.963581,...,45.518883,46.211893,48.452130,53.528258,50.276309,56.656157,48.676531,45.244221,48.159816,46.525829


### 2B) Annotate 

- Run the following to generate a plate list based on Metadata_Plate column.

In [34]:
plate_list = df['Metadata_Plate'].unique().tolist()

In [7]:
print(plate_list)

['220324_134819_Plate_1', '220408_131035_Plate_1']


- All metadata must be organized such:
    ```
    |- metadata 
    |   |   |- <barcode_platemap.csv> 
    |   |- platemap
    |   |   |- <platemap_1.csv>
    |   |   |- <platemap_2.csv>
    ```

- Run the next cell to annotate the profiles. 

In [58]:
df_temp_list = []
for pl in plate_list:
    df_plate = df.loc[df['Metadata_Plate'] == pl]
    #deal with metadata information
    barcode_map = barcode_df[barcode_df['Assay_Plate_Barcode'] == pl]
    index_map = barcode_map.index[0]
    metadata_filename = barcode_df['Plate_Map_Name'][index_map]
    metadata = pd.read_csv(platemap_path + r'/platemap/' + metadata_filename + '.csv')
    #annotate
    df_temp = pycytominer.annotate(profiles = df_plate, platemap = metadata, join_on =  ["Metadata_well_position", "Metadata_Well"])
    df_temp_list.append(df_temp)
    print('Shape of each plate ', df_temp.shape)
df = pd.concat(df_temp_list, axis=0)

Shape of each plate  (12, 1378)
Shape of each plate  (17, 1378)


### 2C) Normalize

- Normalize the dataset, per-plate basis, to **samples = all** or **samples = negcon**.
- CHOOSE one or the other below.

#### Normalize TO NEGCON => run next cell

In [59]:
df_temp_list = []
for pl in plate_list:
    df_temp = df.loc[df['Metadata_Plate'] == pl]
    print(df_temp.shape)
    df_norm_temp = pycytominer.normalize(df_temp, method = 'mad_robustize', mad_robustize_epsilon = 0, samples = "Metadata_control_type == 'negcon'") 
    df_temp_list.append(df_norm_temp)
df_norm2 = pd.concat(df_temp_list, axis=0)
cells_that_run.append('normalized_negcon')

(12, 1378)
(17, 1378)


#### Normalize TO ALL => run next cell

In [47]:
df_temp_list = []
for pl in plate_list:
    df_temp = df.loc[df['Metadata_Plate'] == pl]
    print(df_temp.shape)
    df_norm_temp = pycytominer.normalize(df_temp, method = 'mad_robustize', mad_robustize_epsilon = 0) 
    df_temp_list.append(df_norm_temp)
df_norm2 = pd.concat(df_temp_list, axis=0)
cells_that_run.append('normalized')

(12, 1378)
(17, 1378)


### 2D) Feature selection

In [37]:
keep = ['Cells_Intensity_IntegratedIntensity_OrigLT', 'Cytoplasm_Intensity_IntegratedIntensity_OrigLT', 
        'Cytoplasm_Intensity_MeanIntensity_OrigLT', 'Cells_Intensity_MeanIntensity_OrigLT']

In [60]:
df_keep = df_norm2[keep]

In [61]:
df_selected = pycytominer.feature_select(df_norm2, operation = ['correlation_threshold', 'variance_threshold', 'drop_na_columns','drop_outliers'], outlier_cutoff = 500) 
print('Number of columns removed:', df_norm2.shape[1] - df_selected.shape[1])
print('Percentage of columns removed:',100 - ((df_selected.shape[1]*100)/df_norm2.shape[1]))
cells_that_run.append('feature_select')

Number of columns removed: 1045
Percentage of columns removed: 75.8345428156749


In [62]:
df_selected = pd.concat([df_selected, df_keep], axis='columns')

### 2E) Merge object count

In [63]:
df_selected = object_count.merge(df_selected, on=['Metadata_Plate','Metadata_Well'])

# Export

In [42]:
output_path = eg.diropenbox(msg="Choose an output folder", default=r"D:")
print('Path to save the profile', output_path)

Path to save the profile D:\2022_09_09_LiveCellPainting_fossa_Cimini\workspace\profiles\2022_03_24_Acidification_LT


In [64]:
output_name = '_'.join(cells_that_run)
df_selected.to_csv(output_path + r'/' + output_name + '.csv')
print('Successfully exported to:', output_path + r'/' + output_name + '.csv')

Successfully exported to: D:\2022_09_09_LiveCellPainting_fossa_Cimini\workspace\profiles\2022_03_24_Acidification_LT/2022_03_24_Acidification_LT_normalized_negcon_feature_select_normalized_feature_select_feature_select_feature_select_normalized_negcon_feature_select.csv
